In [ ]:
import pandas as pd
import os

os.chdir(r'C:\Users\jf79\Cloned Repos\REAP-Scripts')
import myfuncs.myfuncs as mf


cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data'
os.chdir(cwd)
mf.read_directory()

master_data = pd.read_csv('Netcall Interactions Master Data.csv',low_memory=False)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry
Files in: ['.csv', 'aggregated_interactions.csv', 'aggregated_interactions.xlsx', 'Footfall Data - Strategic Assessment', 'footfall_data.csv', 'hf_footfall_data.csv', 'london_footfall_data.csv', 'Netcall Dashboard Data.csv', 'Netcall Interactions Master Data.csv', 'Netcall Interactions Master Data.zip', 'residents.csv', 'typical_borough_averages.csv', 'typical_hf (Q3).csv', 'typical_ward_averages.csv', 'visitors.csv', 'weekday_lon (annual).csv', 'weekend_lon (annual).csv', 'workers.csv', 'x.csv']



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

master_df = master_data.copy()

columns_to_drop = [
    'customername','assistedby',
    'casereference','ipaddress'
]
master_df.drop(columns=columns_to_drop, inplace=True)

master_df['nullcount'] = master_df.isna().sum(axis=1)
master_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(drop.index, inplace=True)
master_df.drop(columns='nullcount', inplace=True)

columns_to_change = ['timeended', 'timequeued']
for column in columns_to_change:
    master_df[column] = pd.to_datetime(master_df[column])

master_df['queuetime'] = master_df['timeended'] - master_df['timequeued']

columns_to_change = [
    'duration','talktime','holdtime',
    'wrapupduration','agenthandlingtime'
]
master_df[columns_to_change] = master_df[columns_to_change].apply(lambda x: x.str.replace(r'(?<!^)-', '', regex=True))
for column in columns_to_change:
    master_df[column] = pd.to_timedelta(master_df[column])


master_df['dateofqueue'] = master_df['timequeued'].dt.date
master_df.loc[(master_df['type'] == 'Call') & (master_df['queuetime'] < pd.Timedelta(seconds=40)),'isunder40s?'] = 1
master_df['isunder40s?'].fillna(0, inplace=True)
master_df

C:\Users\jf79\AppData\Local\Temp\ipykernel_3516\1762658216.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df['isunder40s?'].fillna(0, inplace=True)


,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customercontact,queuetime,duration,talktime,holdtime,wrapupduration,agenthandlingtime,result,activitycode,transferredto,subject,conversationstarttime,conversationendtime,conversationduration,agentusername,agentid,forwarded,source,conversationid,Year,Month,dateofqueue,isunder40s?
0,2640671,Repair CSC,Email,Inbound,2022-01-01 02:15:00,2022-01-04 16:01:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,Gbenga.Osunkoya@lbhf.gov.uk,3 days 13:46:00,0 days 00:04:05,0 days 00:04:05,0 days 00:00:00,0 days 00:00:04,0 days 00:04:09,Replied,Repair Status,NaN,RE: Inspection item Requires Attention - (BLK) 1-180 Poynter House,01/01/2022 02:15,04/01/2022 16:01,85:46:28,4861,4861.0,0.0,Email,2516337.0,2022,1,2022-01-01,0.0
1485943,2640673,Customer Services,Email,Inbound,2022-01-01 03:05:38,2022-01-04 09:09:58,Electoral Services,Electoral Services Email,Gillian Hoyle,mccatigan@gmail.com,3 days 06:04:20,0 days 00:00:23,0 days 00:00:22,0 days 00:00:01,0 days 00:00:00,0 days 00:00:22,Other,NaN,NaN,Register to Vote,01/01/2022 03:05:38,04/01/2022 09:09:58,78:04:20,NaN,NaN,1.0,NaN,2516339.0,2022,1,2022-01-01,0.0
1,2640679,Repair CSC,Email,Inbound,2022-01-01 06:14:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Georgia St Rose,Baboucarr.Gaye@lbhf.gov.uk,3 days 09:51:00,0 days 00:03:35,0 days 00:03:34,0 days 00:00:01,0 days 00:00:09,0 days 00:03:43,Replied,Repair Status,NaN,RE: Norland House Electrical Failure (Job Ref. 1126163/1 emergency within 4hrs),01/01/2022 06:14,04/01/2022 16:05,81:51:20,4861,4861.0,0.0,Email,2516345.0,2022,1,2022-01-01,0.0
2,2640681,Repair CSC,Email,Inbound,2022-01-01 09:08:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Emma Hicks,orbisreports@orbisprotect.com,3 days 06:57:00,0 days 00:03:06,0 days 00:03:06,0 days 00:00:00,0 days 00:00:16,0 days 00:03:22,Transferred,OOH - Related,Mehul Patel,[Orbis Protect] - Unallocated Cases,01/01/2022 09:08,04/01/2022 16:17,79:09:21,971,971.0,0.0,Email,2516347.0,2022,1,2022-01-01,0.0
3,2640683,Repair CSC,Email,Inbound,2022-01-01 09:38:00,2022-01-04 16:19:00,Repairs customer e-mail,E-Mail Responses - Customer,Emma Hicks,noreply@lbhf.gov.uk,3 days 06:41:00,0 days 00:14:11,0 days 00:14:10,0 days 00:00:01,0 days 00:01:00,0 days 00:15:10,Replied,Repair Status,NaN,Form submission from: Housing repair notification form,01/01/2022 09:38,04/01/2022 16:19,78:41:00,971,971.0,0.0,Email,2516349.0,2022,1,2022-01-01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450892,7789413,Customer Services,Call,Inbound,2025-03-07 17:15:43,2025-03-07 17:18:39,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,07513132149,0 days 00:02:56,0 days 00:02:45,0 days 00:02:45,0 days 00:00:00,0 days 00:00:30,0 days 00:03:15,Answered,Food Bank Referral,NaN,NaN,07/03/2025 17:15:43,07/03/2025 17:18:39,00:02:56,NaN,NaN,NaN,NaN,7418617.0,2025,3,2025-03-07,0.0
2450893,7789417,Customer Services,Call,Inbound,2025-03-07 17:19:01,2025-03-07 17:22:58,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,02087496008,0 days 00:03:57,0 days 00:03:39,0 days 00:01:04,0 days 00:02:35,0 days 00:00:30,0 days 00:04:09,Answered,Fuel Voucher New Request,NaN,NaN,07/03/2025 17:19:01,07/03/2025 17:22:58,00:03:57,NaN,NaN,NaN,NaN,7418621.0,2025,3,2025-03-07,0.0
2450894,7789433,Customer Services,Call,Outbound,2025-03-07 17:25:36,2025-03-07 17:31:29,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,02087496008,0 days 00:05:53,0 days 00:05:47,0 days 00:05:47,0 days 00:00:00,0 days 00:01:48,0 days 00:07:35,Answered,Fuel Voucher New Request,NaN,NaN,07/03/2025 17:25:36,07/03/2025 17:31:29,00:05:53,NaN,NaN,NaN,NaN,7418637.0,2025,3,2025-03-07,0.0
2450895,7789447,Customer Services,Messaging,Inbound,2025-03-07 17:33:17,2025-03-07 17:35:51,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,NaN,0

In [4]:
aggregated_interactions = master_df.groupby(['direction','type','dateofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('interactionid', 'count'),
    wrap_upduration = ('wrapupduration','mean'),
    queue_time = ('queuetime','mean'),
    duration = ('queuetime','mean'),
    talk_time = ('talktime','mean'),
    hold_time = ('holdtime','mean'),
    agent_handling_time = ('agenthandlingtime','mean'),
    isunder40s = ('isunder40s?','sum')
).reset_index()

# Function to format timedelta
def format_timedelta(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{days:02}.{hours:02}:{minutes:02}:{seconds:02}'
 
columns_to_change = [
    'wrap_upduration','queue_time','duration',
    'talk_time','hold_time','agent_handling_time'
]
for column in columns_to_change:
    aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
    aggregated_interactions[column] = aggregated_interactions[column].apply(format_timedelta)

mf.validate_data(aggregated_interactions)

C:\Users\jf79\AppData\Local\Temp\ipykernel_3516\4182691313.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
C:\Users\jf79\AppData\Local\Temp\ipykernel_3516\4182691313.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

#########################################################################################################################################################################################
DataFrame: None


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time,isunder40s
0,Inbound,Call,2022-01-04,ACE and Admissions,ACE and Admissions,Answered,15,00.00:04:42,00.00:06:12,00.00:06:12,00.00:05:56,00.00:00:00,00.00:10:39,1.0
1,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Abandoned,2,00.00:00:00,00.00:02:00,00.00:02:00,00.00:00:00,00.00:00:00,00.00:00:00,1.0
2,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Answered,66,00.00:00:29,00.00:06:22,00.00:06:22,00.00:05:58,00.00:00:00,00.00:06:28,0.0
3,Inbound,Call,2022-01-04,CAS Team 1 6945,CAS Team 1 6945 Queue,Answered,3,00.00:01:00,00.00:01:40,00.00:01:40,00.00:01:10,00.00:00:00,00.00:02:10,0.0
4,Inbound,Call,2022-01-04,CAS Team 2 6946,CAS Team 2 6946 Queue,Answered,3,00.00:01:00,00.00:02:00,00.00:02:00,00.00:01:38,00.00:00:00,00.00:02:38,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195012,Outbound,Email,2025-03-06,Customer Services,Cost of Living Crisis Email,Deleted,2,00.00:01:00,00.00:00:54,00.00:00:54,00.00:00:52,00.00:00:00,00.00:01:53,0.0
195013,Outbound,Email,2025-03-06,Customer Services,Cost of Living Crisis Email,Sent,4,00.00:01:00,00.00:01:41,00.00:01:41,00.00:01:40,00.00:00:00,00.00:02:40,0.0
195014,Outbound,Email,2025-03-07,Customer Services,Cost of Living Crisis Email,Deleted,2,00.00:01:00,00.00:01:35,00.00:01:35,00.00:01:35,00.00:00:00,00.00:02:35,0.0
195015,Outbound,Email,2025-03-07,Customer Services,Cost of Living Crisis Email,Sent,2,00.00:00:35,00.00:02:05,00.00:02:05,00.00:02:04,00.00:00:00,00.00:02:40,0.0


Unique values per field:


,Field Name,No. of Unique Values
0,direction,2
1,type,3
2,dateofqueue,1155
3,groupname,47
4,queue,186
5,result,11
6,number_of_interactions,425
7,wrap_upduration,949
8,queue_time,19476
9,duration,19476



Number of duplicate rows:
0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195017 entries, 0 to 195016
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   direction               195017 non-null  object 
 1   type                    195017 non-null  object 
 2   dateofqueue             195017 non-null  object 
 3   groupname               195017 non-null  object 
 4   queue                   195017 non-null  object 
 5   result                  195017 non-null  object 
 6   number_of_interactions  195017 non-null  int64  
 7   wrap_upduration         195017 non-null  object 
 8   queue_time              195017 non-null  object 
 9   duration                195017 non-null  object 
 10  talk_time               195017 non-null  object 
 11  hold_time               195017 non-null  object 
 12  agent_handling_time     195017 non-null  object 
 13  isunder40s              195017 non-null  fl

None


Summary statistics:


,number_of_interactions,isunder40s
count,195017.000000,195017.000000
mean,12.311373,1.344232
std,33.608540,22.483909
min,1.000000,0.000000
25%,1.000000,0.000000
50%,4.000000,0.000000
75%,11.000000,1.000000
max,4154.000000,4154.000000


End of data validation
#########################################################################################################################################################################################



In [ ]:
mf.export_to_csv(
    aggregated_interactions,
    directory = r'C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data',
    df_name='Netcall Dashboard Data')

Exproting Netcall Dashboard Data to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\Netcall Dashboard Data.csv

Successfully exported Netcall Dashboard Data to CSV


: 